# Main Notebook

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from CSV for X different models
 -  Load BaseBid Predictions from pickels for X different models
 -  For X models combine (in Y different ways) the above two steps to build X*Y BidPrices for each impression
 -  Hold auction for each impression one by one
 -  Summary Scores for each of X*Y models 

# Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer
import datetime

%matplotlib inline

# Pull Validation Data

In [ ]:
path = "C:/Users/Akis/OneDrive/Masters/Web Economics/Data/"
#test_df = pd.read_csv(path+'test.csv')
#train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')

In [ ]:
#print(validation_df.head())
#avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
#print(avgCTR)

# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [ ]:
#path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/click_predictions_achilleas/'
path='C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'

lr_data = pd.read_csv(path+'lr_pCTR.csv')
lr_pCTR=lr_data[['click_proba']]
lr_pCTR = lr_pCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(lr_pCTR.head(2))

xgb_data = pd.read_csv(path+'xgb_pCTR.csv')
xgb_pCTR=xgb_data[['click_proba']]
xgb_pCTR = lr_pCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(lr_pCTR.head(2))


#path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/click_predictions_dwane/'

mF_data = pd.read_csv(path+'click_predictions_narrow.csv')
mF_pCTR=mF_data[['pCTR']]
#mF_pCTR = mF_pCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(mF_pCTR.head(2))

In [ ]:
#resultCTR=teamAdf
#resultCTR["teamB_CTR"]=teamBdf["CTR"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
#print(resultCTR.head())


# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [ ]:
#path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/basebid_predictions_achilleas/'
path='C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'

r_data = pd.read_csv(path+'ridge_basebid.csv')
r_pBB = r_data[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(r_pBB.head(2))

lasso_basebid = pd.read_csv(path+'lasso_basebid.csv')
lasso_pBB = lasso_basebid[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(lasso_pBB.head(2))

elasticnet_basebid = pd.read_csv(path+'elasticnet_basebid.csv')
eN_pBB = elasticnet_basebid[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(eN_pBB.head(2))

rfr_basebid = pd.read_csv(path+'rfg_basebid.csv')
rfr_pBB = rfr_basebid[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(rfr_pBB.head(2))

EightyNineBB=r_pBB * 0 + 89

# Formulas to use

In [ ]:
def CostantBidding(resBaseBid,resCTR):
    Bids=resBaseBid['predicted_payprice'].values * 0 + 79
    #print("Bids:",Bids)
    return Bids

In [ ]:
def RandomBidding(resBaseBid,resCTR):
    Bids = np.random.randint(69, 89+1, resBaseBid.shape[0])
    return Bids

In [ ]:
def BestLinearBidding(resBaseBid,resCTR):
    avCTR = resCTR['pCTR'].mean()
    Bids = resBaseBid['predicted_payprice'].values * (resCTR['pCTR'].values/avCTR)
    return Bids

In [ ]:
def Strategy1(sub_basebid,sub_pCTR):
    avgCTR = sub_pCTR["pCTR"].mean()
    base_bid = 105.46
    bidprice = (sub_pCTR["pCTR"] * base_bid) / avgCTR
    return bidprice

In [ ]:
def Strategy2(sub_basebid,sub_pCTR):
    avgCTR = sub_pCTR["pCTR"].mean()
    bidprice = (sub_pCTR["pCTR"].values * sub_basebid["predicted_payprice"].values) / avgCTR
    return bidprice

In [ ]:
# def Function1(resBaseBid,resCTR):
#     avCTR = 0.5
#     Bids=resBaseBid['base_bid'].values * (resCTR['pCTR'].values/avCTR)
#     #print("Bids:",Bids)
#     return Bids

In [ ]:
# def Function2(reBaseBid,reCTR):
#     avCTR = 0.5
#     reCTR=reCTR.copy(True)
#     reCTR=(((reCTR+0.5)**2)-0.5)
#     reCTR[reCTR < 0] = 0
#     Bids=reBaseBid.values * (reCTR/avCTR)
#     return Bids

In [ ]:
# def Function3(rBaseBid,rCTR):
#     avCTR = 0.5
#     rCTR=rCTR.copy(True)
#     rCTR[rCTR < 0.5] = 0
#     #rCTR=(((rCTR+0.5)**4)-0.5)
#     Bids=rBaseBid.values*1.2 * (rCTR/avCTR)
#     return Bids

# Build Bid Prices for all models

Need a function that will build a new dataframe conatining bids of all teams by combining the previous two dataframes.

In [ ]:
resultsBids=pd.DataFrame(index=range(validation_df.shape[0]))
resultsBids['click']=validation_df['click']
resultsBids['payprice']=validation_df['payprice']

# resultsBids['ConstantBidding'] = CostantBidding(EightyNineBB,lr_data)
# resultsBids['RandomBidding'] = RandomBidding(EightyNineBB,lr_data)
# resultsBids['BestLinear'] = BestLinearBidding(EightyNineBB,xgb_pCTR)
# resultsBids['xgb_lasso_Str2'] = Strategy2(lasso_pBB,xgb_pCTR)
# resultsBids['mF_rfr_Str2'] = Strategy2(rfr_pBB,mF_pCTR)
# resultsBids['mF_eN_Str1'] = Strategy1(eN_pBB,mF_pCTR)

# multiples = 7
# resultsBids['ConstantBidding'] = resultsBids['ConstantBidding'].values * multiples
# resultsBids['RandomBidding'] = resultsBids['RandomBidding'].values * multiples
# resultsBids['BestLinear'] = resultsBids['BestLinear'].values * multiples
# resultsBids['xgb_lasso_Str2'] = resultsBids['xgb_lasso_Str2'].values * multiples
# resultsBids['mF_rfr_Str2'] = resultsBids['mF_rfr_Str2'].values * multiples
# resultsBids['mF_eN_Str1'] = resultsBids['mF_eN_Str1'].values * multiples

In [ ]:
df_summary = pd.DataFrame(columns=['budget','click','team_name','win','multiples'])

In [ ]:
df_summary

In [ ]:
for multiples in range(40,81,2):
    print(int((multiples-38)/2),"/ 20 .....",)
    print(datetime.datetime.now().time())
    resultsBids['ConstantBidding'] = CostantBidding(EightyNineBB,lr_data)
    resultsBids['RandomBidding'] = RandomBidding(EightyNineBB,lr_data)
    resultsBids['BestLinear'] = BestLinearBidding(EightyNineBB,xgb_pCTR)
    resultsBids['xgb_lasso_Str2'] = Strategy2(lasso_pBB,xgb_pCTR)
    resultsBids['mF_rfr_Str2'] = Strategy2(rfr_pBB,mF_pCTR)
    resultsBids['mF_eN_Str1'] = Strategy1(eN_pBB,mF_pCTR)

    resultsBids['ConstantBidding'] = resultsBids['ConstantBidding'].values * multiples/10
    resultsBids['RandomBidding'] = resultsBids['RandomBidding'].values * multiples/10
    resultsBids['BestLinear'] = resultsBids['BestLinear'].values * multiples/10
    resultsBids['xgb_lasso_Str2'] = resultsBids['xgb_lasso_Str2'].values * multiples/10
    resultsBids['mF_rfr_Str2'] = resultsBids['mF_rfr_Str2'].values * multiples/10
    resultsBids['mF_eN_Str1'] = resultsBids['mF_eN_Str1'].values * multiples/10

    dataSubset = resultsBids.sample(frac=0.2, replace=False).copy(True) #divided by 5 to get similar environment
    dataSubset.reset_index(drop=True, inplace=True)

    s = Scorer()
    df_OneIteration,df = s.set_df(dataSubset, ['ConstantBidding', 'RandomBidding', 'BestLinear','xgb_lasso_Str2', 'mF_rfr_Str2', 'mF_eN_Str1'],6250000,20) 
    # /5 because 6 teams instead of 30
    df_OneIteration['multiples'] = multiples/10
    print("Multiples of budget spent:",multiples/10)
    print(df_OneIteration.sort_values(by=['team_name'])) #.sort_values(by=['click'])
    df_summary = df_summary.append(df_OneIteration)
    print()
    print()

In [ ]:
df_summary.head(10)

In [ ]:
df_summary.to_csv('MultiAgentResults.csv')

In [ ]:
def PlotMultiAgentResults(results,metric):
    x = results['multiples'].sort_values()
    x = x.unique()

    BestLinear = results[metric][results['team_name'] =='BestLinear']
    RandomBidding = results[metric][results['team_name'] =='RandomBidding']
    mF_rfr_Str2 = results[metric][results['team_name'] =='mF_rfr_Str2']
    mF_eN_Str1 = results[metric][results['team_name'] =='mF_eN_Str1']
    xgb_lasso_Str2 = results[metric][results['team_name'] =='xgb_lasso_Str2']
    ConstantBidding = results[metric][results['team_name'] =='ConstantBidding']

    plt.plot(x, BestLinear)
    plt.plot(x, RandomBidding)
    plt.plot(x, mF_rfr_Str2)
    plt.plot(x, mF_eN_Str1)
    plt.plot(x, xgb_lasso_Str2)
    plt.plot(x, ConstantBidding)
    
    plt.title(metric+'s for each strategy')
    plt.xlabel('Multiples of Budget')
    plt.legend(['BestLinear', 'RandomBidding', 'mF_rfr_Str2', 'mF_eN_Str1', 'xgb_lasso_Str2', 'ConstantBidding'], loc='best')

    plt.show()

In [ ]:
PlotMultiAgentResults(df_summary,'win')

In [ ]:
PlotMultiAgentResults(df_summary,'click')

In [ ]:
PlotMultiAgentResults(df_summary,'budget')

In [ ]:
# for multiples in range(150,160,5):
#     resultsBids['ConstantBidding'] = CostantBidding(EightyNineBB,lr_data)
#     resultsBids['RandomBidding'] = RandomBidding(EightyNineBB,lr_data)
#     resultsBids['BestLinear'] = BestLinearBidding(EightyNineBB,xgb_pCTR)
#     resultsBids['xgb_lasso_Str2'] = Strategy2(lasso_pBB,xgb_pCTR)
#     resultsBids['mF_rfr_Str2'] = Strategy2(rfr_pBB,mF_pCTR)
#     resultsBids['mF_eN_Str1'] = Strategy1(eN_pBB,mF_pCTR)

#     resultsBids['ConstantBidding'] = resultsBids['ConstantBidding'].values * multiples/10
#     resultsBids['RandomBidding'] = resultsBids['RandomBidding'].values * multiples/10
#     resultsBids['BestLinear'] = resultsBids['BestLinear'].values * multiples/10
#     resultsBids['xgb_lasso_Str2'] = resultsBids['xgb_lasso_Str2'].values * multiples/10
#     resultsBids['mF_rfr_Str2'] = resultsBids['mF_rfr_Str2'].values * multiples/10
#     resultsBids['mF_eN_Str1'] = resultsBids['mF_eN_Str1'].values * multiples/10

#     dataSubset = resultsBids.sample(frac=0.002, replace=False).copy(True) #divided by 5 to get similar environment
#     dataSubset.reset_index(drop=True, inplace=True)

#     s = Scorer()
#     df_summary,df = s.set_df(dataSubset, ['ConstantBidding', 'RandomBidding', 'BestLinear','xgb_lasso_Str2', 'mF_rfr_Str2', 'mF_eN_Str1'],6250000,20) 
#     # /5 because 6 teams instead of 30
#     print("Multiples of budget spent:",multiples/10)
#     print(df_summary.sort_values(by=['click']))
#     print()
#     print()

In [ ]:
resultsBids.head(20)

In [ ]:
dataSubset = resultsBids.sample(frac=0.2, replace=False).copy(True) #divided by 5 to get similar environment
dataSubset.reset_index(drop=True, inplace=True)

In [ ]:
dataSubset.head(10)

# Hold Auction for each

Need a function that will step through the impressions one by one checking who won the impression, whilst keeping track of each player's budget.

In [ ]:
s = Scorer()
df_summary,df = s.set_df(dataSubset, ['ConstantBidding', 'RandomBidding', 'BestLinear','xgb_lasso_Str2', 'mF_rfr_Str2', 'mF_eN_Str1'],6250000,20) 
# /5 because 6 teams instead of 30

# Scores

Summarise the main metrics for each team.

In [ ]:
df_summary['CTR']=df_summary['click']/df_summary['win']
print(df_summary)

In [ ]:
df_summary['win'].sum()/(validation_df.shape[0]*0.2)

In [ ]:
#print(df)